# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of movie reviews, accompanied by labels.

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=400px>

Here, we'll pass in words to an embedding layer. We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the word2vec lesson. You can actually train up an embedding with word2vec and use it here. But it's good enough to just have an embedding layer and let the network learn the embedding table on it's own.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. We're using the sigmoid because we're trying to predict if this text has positive or negative sentiment. The output layer will just be a single unit then, with a sigmoid activation function.

We don't care about the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

In [3]:
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

## Data preprocessing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. We'll want to get rid of those periods. Also, you might notice that the reviews are delimited with newlines `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [4]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [5]:
all_text[:2000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [6]:
words[:100]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 'high',
 's',
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 'teachers',
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

In [7]:
# Create your dictionary that maps vocab words to integers here
uniqueWords = set(words)
vocab_to_int = {val:i+1 for i, val in enumerate(uniqueWords)}

# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = [[vocab_to_int[w] for w in k.split()] for k in reviews]

In [8]:
len(uniqueWords)

74072

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

In [9]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = [1 if val=="positive" else 0 for val in labels.split("\n")]

If you built `labels` correctly, you should see the next output.

In [10]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. Let's truncate to 200 steps. For reviews shorter than 200, we'll pad with 0s. For reviews longer than 200, we can truncate them to the first 200 words.

In [11]:
# Filter out that review with 0 length
nonZeroIndeces = []
for i, review in enumerate(reviews_ints):
    if(len(review)!=0):
        nonZeroIndeces.append(i)

reviews_ints = [reviews_ints[i] for i in nonZeroIndeces]
labels = [[labels[i]] for i in nonZeroIndeces]

In [12]:
seq_len = 200
features = []
for review in reviews_ints:
    if(len(review)>seq_len):
        features.append(review[:seq_len])
    else:
        features.append([0]*(seq_len-len(review))+review)

features = np.array(features)

If you build features correctly, it should look like that cell output below.

In [13]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 38204, 57019, 57862,
         5730, 12185, 16267, 62442, 23812, 14853, 15010,  9741, 29041,
         2426, 66797, 44939, 46332, 42709, 59573,  7774, 15555,  2426,
        30990, 42657, 25686, 42500, 15010,   619, 65720, 50545, 63435,
         7023, 13706,  2471, 38204, 57019, 37947, 22497, 57862, 46439,
        59965],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

In [14]:
split_frac = 0.8

train_x, val_x = features[:int(0.8*len(features))], features[int(0.8*len(features)):]
train_y, val_y = labels[:int(0.8*len(features))], labels[int(0.8*len(features)):]

val_x, test_x = val_x[:int(0.5*len(val_x))], val_x[int(0.5*len(val_x)):]
val_y, test_y = val_y[:int(0.5*len(val_y))], val_y[int(0.5*len(val_y)):]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [15]:
len(val_y)

2500

With train, validation, and text fractions of 0.8, 0.1, 0.1, the final shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

For the network itself, we'll be passing in our 200 element long review vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

### LSTM cell

<img src="assets/network_diagram.png" width=400px>

Next, we'll create our LSTM cells to use in the recurrent network ([TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)). Here we are just defining what the cells look like. This isn't actually building the graph, just defining the type of cells we want in our graph.

To create a basic LSTM cell for the graph, you'll want to use `tf.contrib.rnn.BasicLSTMCell`. Looking at the function documentation:

```
tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0, input_size=None, state_is_tuple=True, activation=<function tanh at 0x109f1ef28>)
```

you can see it takes a parameter called `num_units`, the number of units in the cell, called `lstm_size` in this code. So then, you can write something like 

```
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

to create an LSTM cell with `num_units`. Next, you can add dropout to the cell with `tf.contrib.rnn.DropoutWrapper`. This just wraps the cell in another cell, but with dropout added to the inputs and/or outputs. It's a really convenient way to make your network better with almost no effort! So you'd do something like

```
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
```

Most of the time, your network will have better performance with more layers. That's sort of the magic of deep learning, adding more layers allows the network to learn really complex relationships. Again, there is a simple way to create multiple layers of LSTM cells with `tf.contrib.rnn.MultiRNNCell`:

```
cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
```

Here, `[drop] * lstm_layers` creates a list of cells (`drop`) that is `lstm_layers` long. The `MultiRNNCell` wrapper builds this into multiple layers of RNN cells, one for each cell in the list.

So the final cell you're using in the network is actually multiple (or just one) LSTM cells with dropout. But it all works the same from an architectural viewpoint, just a more complicated graph in the cell.

Here is [a tutorial on building RNNs](https://www.tensorflow.org/tutorials/recurrent) that will help you out.


### RNN forward pass

<img src="assets/network_diagram.png" width=400px>


### Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with `outputs[:, -1]`, the calculate the cost from that and `labels_`.

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [16]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. Before you run this, make sure the `checkpoints` directory exists.

In [21]:
epochs = 10
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001
n_words = len(uniqueWords)+1
optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)

In [22]:
from tensorflow.keras import layers, Model


inputLayer = layers.Input(shape=(None,))
embeddingLayer = layers.Embedding(n_words, 100)(inputLayer)
lstms = []
for i in range(lstm_layers):
    lstms.append(layers.LSTMCell(lstm_size, dropout=0.5))
lstmLayer = layers.RNN(lstms)(embeddingLayer)
denseLayer = layers.Dense(1, activation=tf.sigmoid)(lstmLayer)

sentimentModel = Model(inputs=inputLayer, outputs=denseLayer, name="Sentiment Classifier")

sentimentModel.summary()

Model: "Sentiment Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         7407300   
_________________________________________________________________
rnn_1 (RNN)                  (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 7,773,125
Trainable params: 7,773,125
Non-trainable params: 0
_________________________________________________________________


In [23]:
for i in range(epochs):

    for x, y in get_batches(train_x, train_y, batch_size=500):

        with tf.GradientTape() as tape:
            predictions = sentimentModel(x, training=True)
            loss = tf.losses.mean_squared_error(y, predictions)
            
        grads = tape.gradient(loss, sentimentModel.trainable_variables)
        optimizer.apply_gradients(zip(grads, sentimentModel.trainable_variables))
        
    
    validationPreds = sentimentModel(val_x)
    correct_pred = tf.equal(tf.cast(tf.round(validationPreds), tf.int32), val_y)
    accuracy = tf.reduce_sum(tf.cast(correct_pred, tf.float32))/len(val_x)
    print(f"At the epoch {i+1} with loss {tf.reduce_mean(loss)} accuracy for validation: {accuracy}")

At the epoch 1 with loss 0.36065298318862915 accuracy for validation: 0.6908000111579895
At the epoch 2 with loss 0.20308811962604523 accuracy for validation: 0.724399983882904
At the epoch 3 with loss 0.15966880321502686 accuracy for validation: 0.8091999888420105
At the epoch 4 with loss 0.10281676799058914 accuracy for validation: 0.7832000255584717
At the epoch 5 with loss 0.14983798563480377 accuracy for validation: 0.7148000001907349
At the epoch 6 with loss 0.0937216505408287 accuracy for validation: 0.7663999795913696
At the epoch 7 with loss 0.08702699840068817 accuracy for validation: 0.717199981212616
At the epoch 8 with loss 0.10908479243516922 accuracy for validation: 0.8104000091552734
At the epoch 9 with loss 0.05572158470749855 accuracy for validation: 0.8091999888420105
At the epoch 10 with loss 0.04669271409511566 accuracy for validation: 0.7835999727249146


## Testing

In [24]:
testPreds = sentimentModel(test_x)
correct_pred = tf.equal(tf.cast(tf.round(testPreds), tf.int32), test_y)
accuracy = tf.reduce_sum(tf.cast(correct_pred, tf.float32))/len(test_y)
print(f"Accuracy for Test Set: {accuracy}")

Accuracy for Test Set: 0.7972000241279602
